In [1]:
import pandas as pd
import numpy as np
import pickle

import aidds_buy.sys.config as cfg
from aidds_buy.sys.utils.exception import AiddsException
from aidds_buy.sys.app_init import AiddsInit, PltInit
from aidds_buy.sys.utils.data_io import read_data
from aidds_buy.sys.utils.evaluations import regression_evals

In [2]:
AiddsInit()
PltInit()

In [3]:
try:
    model = {
        pc_key: read_data(file_code=f'DUMP,MODELS,{pc_key},BEST') \
            for pc_key in cfg.PC_TYPEs
    }
    data = {
        pc_key: {
            'X': read_data(file_code=f'SCALING,TEST_X,{pc_key}'),
            'y': read_data(file_code=f'SCALING,TEST_y,{pc_key}'),
        } for pc_key in cfg.PC_TYPEs
    }
except AiddsException as ae:
    ae.print()
except Exception as e:
    print(e)

In [4]:
pred_y = {}
for pc_key in cfg.PC_TYPEs:
    pred_y[pc_key] = model[pc_key].predict(data[pc_key]['X'])

In [5]:
def evals(pc_key=None, y=data, p=pred_y):
    test_y = data[pc_key]['y'].to_numpy().reshape(-1)
    _, message = regression_evals(y=test_y, p=pred_y[pc_key], verbose=1)
    print(f'[{pc_key}]: {message}')
    

In [6]:
for pc_key in cfg.PC_TYPEs:
    evals(pc_key)

[ALL]: MAPE: 0.602591(60.2591), R2SCORE: 0.415211
[1]: MAPE: 1.233139(123.3139), R2SCORE: -1.653789
[N1]: MAPE: 0.158091(15.8091), R2SCORE: 0.712867


In [10]:
def check_predict(pc_key=None, idx=None, y=data, p=pred_y):
    test_y = data[pc_key]['y'].to_numpy().reshape(-1)[idx]
    pred_y = p[pc_key][idx]
    print(test_y, pred_y)

In [12]:
for i in range(10):
    check_predict('ALL', i)

2225828 3678852.494674758
3368623 6611927.480750836
2659772 4214924.119257597
6371130 7354762.223163698
1181308 3774336.6875937455
2834296 4250861.079774442
2272593 3742998.2283330313
2438272 3880429.230356717
3259617 4240698.877921393
2053539 4050821.661220757
